In [1]:
import pandas as pd
import numpy as np

from energyanalysis.data_classes import Building, BuildingData, HeatingSystem, Address, Results
from energyanalysis.plots import plot_hourly_series, plot_monthly_series, double_axis_monthly_plot, plot_npv
from energyanalysis.demand import Demand
from energyanalysis.geoenergy import GeoEnergy
from energyanalysis.air_water_heatpump import AirWaterHeatPump
from energyanalysis.district_heating import DistrictHeating
from energyanalysis.solar_panels import SolarPanels
from energyanalysis.electric_vehicle import ElectricVehicle
from energyanalysis.utilities import building_compilation, hour_to_month
from energyanalysis.operation_costs import OperationCosts
from energyanalysis.nnv_analysis import NetPresentValue


In [45]:
df_temperature = pd.read_excel('src/data/temperature_sheet_trondheim.xlsx')
outdoor_temperature = list(df_temperature['Trondheim (N3031)'])
print(outdoor_temperature)

[2.7, 1.93, 1.16, 0.39, -0.38, -1.15, -1.92, -2.69, -0.77, -0.62, -1.72, -1.77, -1.02, -1.37, -1.07, -0.57, -0.92, -0.72, -0.87, -0.87, -0.67, -0.37, -0.32, -0.52, -0.42, -0.67, 0.13, 0.13, 0.23, 0.48, 0.28, -0.47, -0.22, 0.28, -0.22, 0.03, 0.73, 0.43, 1.08, 0.58, 0.53, 0.38, 0.13, 0.58, 1.83, 0.53, 0.68, 0.33, -0.22, 0.08, 0.48, 0.98, 1.08, 0.53, -0.02, -0.37, -0.47, -0.42, -0.92, -1.52, -1.62, -2.32, -3.27, -4.02, -5.47, -6.12, -6.67, -7.22, -7.47, -7.57, -8.42, -8.57, -8.67, -9.22, -9.32, -9.42, -10.02, -9.57, -9.17, -9.12, -8.32, -7.52, -7.37, -6.92, -6.52, -7.27, -7.12, -6.67, -6.52, -6.32, -6.02, -5.42, -5.32, -4.87, -4.97, -4.77, -4.27, -3.97, -3.37, -2.77, -2.92, -2.22, -1.77, -1.52, -1.97, -1.67, 0.13, -0.27, 0.48, 0.53, 0.73, 0.93, 0.83, 1.48, 1.68, 1.63, 1.83, 1.78, 1.33, 1.23, 1.33, 1.23, 3.48, 4.58, 4.28, 4.03, 3.88, 1.98, 2.83, 2.88, 0.43, -0.27, -0.52, -0.77, -0.67, -0.42, -0.92, -1.32, -1.57, -1.82, -1.87, -1.67, -1.87, -2.02, -2.17, -2.07, -1.97, -1.77, -1.87, -1.77, -

In [52]:
building = Building(
    data=BuildingData([2898], ['Kontor'], ['Middels energieffektiv'], outdoor_temperature, [100, 100]),
    address=Address('Test', 63, 10),
    results=Results(),
    )

In [53]:
# Demand().profet_api(building, spaceheating_target_sum=1200000, dhw_target_sum=0, electric_target_sum=0)
Demand().profet_api(building)


(array([16.32057644, 16.85852974, 18.00051088, ..., 13.95021373,
        13.53718175, 14.6948243 ]),
 array([2.43774394, 2.4487717 , 2.14256041, ..., 3.1287473 , 2.72893178,
        2.63896549]),
 array([32.42229656, 32.31944944, 32.57269697, ..., 53.58657893,
        51.74461013, 50.39771247]))

In [54]:
print(vars(building.results.energy))

{'spaceheating': array([16.32057644, 16.85852974, 18.00051088, ..., 13.95021373,
       13.53718175, 14.6948243 ]), 'dhw': array([2.43774394, 2.4487717 , 2.14256041, ..., 3.1287473 , 2.72893178,
       2.63896549]), 'elspecific': array([32.42229656, 32.31944944, 32.57269697, ..., 53.58657893,
       51.74461013, 50.39771247]), 'heating': array([18.75832038, 19.30730144, 20.14307129, ..., 17.07896104,
       16.26611353, 17.33378979]), 'cooling': None, 'geoenergy_compressor': None, 'geoenergy_extracted_from_wells': None, 'geoenergy_peak': None, 'geoenergy_electricity': None, 'air_water_heat_pump_compressor': None, 'air_water_heat_pump_extracted_from_air': None, 'air_water_heat_pump_peak': None, 'air_water_heat_pump_electricity': None, 'air_air_heat_pump_compressor': None, 'air_air_heat_pump_extracted_from_air': None, 'air_air_heat_pump_peak': None, 'air_air_heat_pump_electricity': None, 'district_heating': None, 'solar_production': None, 'electric_vehicle': None, 'cooling_covered': None

In [55]:
print(f'Romoppvarmingsbehov: {int(building.results.energy.spaceheating.sum()):,} kWh/år'.replace(',', ' '))
print(f'Tappevannsbehov: {int(building.results.energy.dhw.sum()):,} kWh/år'.replace(',', ' '))
print(f'Elspesifikt behov: {int(building.results.energy.elspecific.sum()):,} kWh/år'.replace(',', ' '))

Romoppvarmingsbehov: 123 558 kWh/år
Tappevannsbehov: 34 769 kWh/år
Elspesifikt behov: 504 377 kWh/år
